In [1]:
# =============================================================================
# PRÁCTICA 4: MINIMIZACION DE FUNCIONES DE n VARIABLES SIN RESTRICCIONES
#
# Objetivo:
# 	Minimizar la función objetivo f(x) (TRID-n modificada) leyendo sus
# 	coeficientes p(j), q(j) y r(j) de un archivo de datos.
#
# 	Función a minimizar (TRID-n modificada):
# 	f(x) = SUMA (p(j)*x(j)**2 + q(j)*x(j), j=1..n) + SUMA (r(j)*x(j)*x(j-1), j=2..n)
#
# Métodos Implementados:
# 	1. Algoritmo del MÁXIMO DESCENSO (Steepest Descent):
# 	2. Algoritmo de NEWTON PROTEGIDO:
# 	3. PYOMO/IPOPT:
#
# Entradas:
# 	- Archivo de entrada: 'Datos4.dat' conteniendo p(j), q(j), r(j).
#
# Salidas:
# 	- Archivo de salida: Solucion4.sol.
# =============================================================================


import numpy as np
import pyomo.environ as pyo

# ======================== LECTURA Y EXTRACCIÓN DE DATOS ======================
def Leer_Datos(nombre_archivo: str) -> tuple:
    
    """
    Lee los coeficientes p(j), q(j) y r(j) para la función objetivo TRID-n.
    """
    
    P_COEF = {}
    Q_COEF = {}
    R_COEF = {}
    
    try:
        with open(nombre_archivo, 'r') as f:
            lineas = f.readlines()
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: El archivo '{nombre_archivo}' no se encontró.")

    # Busca la línea de inicio de la tabla de coeficientes
    inicio_datos = 0
    for i, linea in enumerate(lineas):
        if '---' in linea:
            inicio_datos = i + 1
            break
            
    # Procesa las líneas de datos        
    for linea in lineas[inicio_datos:]:
        if '===' in linea or not linea.strip():
            break
            
        partes = linea.replace('::', ' ').split()
        if not partes:
            continue

        j = int(partes[0])
        coeficientes = [float(c) for c in partes[1:]]
        
        if j == 1:
            P_COEF[j] = coeficientes[0]
            Q_COEF[j] = coeficientes[1]
        else: 
            P_COEF[j] = coeficientes[0]
            Q_COEF[j] = coeficientes[1]
            R_COEF[j] = coeficientes[2]
            
    return P_COEF, Q_COEF, R_COEF, len(P_COEF)


# ==================== CÁLCULO DE FUNCIONES (f, g, G) =======================

def f_trid(x, P_COEF, Q_COEF, R_COEF, N_VARS):
    
    """
    Calcula el valor de la función objetivo TRID-n modificada en el punto x.
    """
 
    f_val = 0.0
    
    for j in range(1, N_VARS + 1):
        f_val += P_COEF[j] * x[j-1]**2 + Q_COEF[j] * x[j-1]
        
    for j in range(2, N_VARS + 1):
        f_val += R_COEF[j] * x[j-1] * x[j-2]
    
    return f_val

def g_trid(x, P_COEF, Q_COEF, R_COEF, N_VARS):

    """
    Calcula el vector gradiente (g = grad(f)) de la función TRID-n.
    """
    
    g_vec = np.zeros(N_VARS)

    # Derivada para la primera variable (j=1)
    g_vec[0] = 2 * P_COEF[1] * x[0] + Q_COEF[1]
    if N_VARS >= 2:
        g_vec[0] += R_COEF[2] * x[1]

    # Derivadas para las variables intermedias
    for j in range(2, N_VARS):
        g_vec[j-1] = 2 * P_COEF[j] * x[j-1] + Q_COEF[j]
        g_vec[j-1] += R_COEF[j] * x[j-2]
        g_vec[j-1] += R_COEF[j+1] * x[j]
        
    # Derivada para la última variable    
    g_vec[N_VARS-1] = 2 * P_COEF[N_VARS] * x[N_VARS-1] + Q_COEF[N_VARS]
    g_vec[N_VARS-1] += R_COEF[N_VARS] * x[N_VARS-2]
    
    return g_vec

def G_trid(P_COEF, R_COEF, N_VARS):

    """
    Construye la matriz Hessiana (G) de la función TRID-n.
    La Hessiana es constante y tridiagonal, ya que f(x) es cuadrática.
    """
    
    G_mat = np.zeros((N_VARS, N_VARS))

    # Elementos de la diagonal principal
    for j in range(1, N_VARS + 1):
        G_mat[j-1, j-1] = 2 * P_COEF[j]
        
    # Elementos fuera de la diagonal
    for j in range(2, N_VARS + 1):
        G_mat[j-1, j-2] = R_COEF[j]
        G_mat[j-2, j-1] = R_COEF[j]
    return G_mat

# ===================== BÚSQUEDA LINEAL DE ARMIJO =====================

def Busqueda_Lineal_Armijo(f, xk, pk, gk, fk, c1=1e-4, rho=0.5):
    
    """
    Implementación del método de búsqueda lineal de Armijo.
    
    Determina un tamaño de paso 'lamda' que garantiza un "descenso suficiente"
    en la función objetivo f(x) a lo largo de la dirección 'pk'.
    """
    
    lamda = 1.0
    descenso = np.dot(gk.T, pk)

    while f(xk + lamda * pk) > fk + c1 * lamda * descenso:
        lamda *= rho
        
        # Criterio de parada forzado si el paso es demasiado pequeño
        if lamda < 1e-16:
            return 0.0

    return lamda


# ===================== ALGORITMO DE MÁXIMO DESCENSO =====================

def Maximo_Descenso(f, g, x0, epsilon_g, max_iter):
    
    """
    Algoritmo del Steepest Descent (Máximo Descenso) con Busqueda_Lineal_Armijo().
    """
    
    x_k = np.array(x0)
    k = 0
    
    # PASO 0
    f_k = f(x_k)
    g_k = g(x_k)
    
    # PASO 1. Criterio de parada
    while np.linalg.norm(g_k) > epsilon_g and k < max_iter:
        
        # PASO 1. Calcular d_k = -grad(f(x_k))
        d_k = -g_k
        
        # PASO 2. Calcular lambda_k (Búsqueda lineal)
        lamda_opt = Busqueda_Lineal_Armijo(f, x_k, d_k, g_k, f_k)
        
        if lamda_opt == 0.0:
            print(f"Advertencia: Búsqueda lineal falló en la iteración {k}.")
            break

        # PASO 2. Poner x_k+1 = x_k + lambda_k * d_k
        x_k_plus_1 = x_k + lamda_opt * d_k
        
        # PASO 3 y 4. Actualizar y seguir
        x_k = x_k_plus_1
        f_k = f(x_k)
        g_k = g(x_k)
        k += 1

    return x_k, f_k, g_k, k


# ===================== ALGORITMO DE NEWTON PROTEGIDO =====================

def Newton_Protegido(f, g, G_mat, x0, epsilon_g, epsilon_p, max_iter, alpha):
    
    """
    Utiliza la dirección de Newton, pero se revierte a Máximo Descenso
    si la dirección de Newton no es de descenso o si la matriz Hessiana
    es singular.
    """
    
    x_k = np.array(x0)
    k = 0
    f_k = f(x_k)
    g_k = g(x_k)
    
    if np.linalg.norm(g_k) < epsilon_g: return x_k, f_k, g_k, k

    while k < max_iter:
        
        # PASO 1: Calcular la dirección de búsqueda p_k
        try:
            p_k = np.linalg.solve(G_mat, -g_k)
            descenso = np.dot(g_k.T, p_k)
            if descenso > -alpha * np.linalg.norm(g_k) * np.linalg.norm(p_k):
                p_k = -g_k
        except np.linalg.LinAlgError:
            p_k = -g_k
        
        # PASO 2: Calcular lambda_k (Búsqueda Lineal de Armijo manual)
        lamda_opt = Busqueda_Lineal_Armijo(f, x_k, p_k, g_k, f_k)
        
        if lamda_opt == 0.0:
            print(f"Advertencia (Newton): Búsqueda lineal falló en la iteración {k}.")
            break

        x_k_plus_1 = x_k + lamda_opt * p_k

        # PASO 3: Actualizar y Criterio de Parada
        f_k_plus_1 = f(x_k_plus_1)
        g_k_plus_1 = g(x_k_plus_1)
        
        if np.linalg.norm(g_k_plus_1) < epsilon_g or np.linalg.norm(x_k_plus_1 - x_k) < epsilon_p:
            return x_k_plus_1, f_k_plus_1, g_k_plus_1, k + 1
            
        x_k = x_k_plus_1
        f_k = f_k_plus_1
        g_k = g_k_plus_1
        k += 1
        
    return x_k, f_k, g_k, k


# ===================== RESOLUCIÓN CON PYOMO/IPOPT =====================

def Resolver_Pyomo(P_COEF, Q_COEF, R_COEF, N_VARS):
    
    """
    Resuelve el problema de minimización usando Pyomo 
    y el solver profesional IPOPT.
    """
    
    model = pyo.ConcreteModel()
    model.J = pyo.RangeSet(1, N_VARS)
    model.x = pyo.Var(model.J, initialize=0.0) 

    def objective_rule(model):
        sum1 = sum(P_COEF[j] * model.x[j]**2 + Q_COEF[j] * model.x[j] for j in model.J)
        sum2 = sum(R_COEF[j] * model.x[j] * model.x[j-1] for j in model.J if j >= 2)
        return sum1 + sum2

    model.OBJ = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    solver = pyo.SolverFactory('ipopt')
    
    results = solver.solve(model, tee=False)

    x_min_pyomo = np.array([pyo.value(model.x[j]) for j in model.J])
    f_min_pyomo = pyo.value(model.OBJ)
    
    g_min_pyomo = g_trid(x_min_pyomo, P_COEF, Q_COEF, R_COEF, N_VARS)
    
    return x_min_pyomo, f_min_pyomo, g_min_pyomo


# ===================== BLOQUE PRINCIPAL DE EJECUCIÓN =====================

def Ejecutar_Escribir(nombre_in, nombre_out, epsilon_g , epsilon_p,  max_iter): 
    try:
        # LECTURA DE DATOS
        P_COEF, Q_COEF, R_COEF, N_VARS = Leer_Datos(nombre_in)
        if N_VARS == 0:
            raise ValueError("No se pudo cargar ningún coeficiente.")
    
        # 7.1. PREPARACIÓN DE WRAPPERS
        f_wrapper = lambda x: f_trid(x, P_COEF, Q_COEF, R_COEF, N_VARS)
        g_wrapper = lambda x: g_trid(x, P_COEF, Q_COEF, R_COEF, N_VARS)
        G_mat = G_trid(P_COEF, R_COEF, N_VARS)
    
        x0 = np.zeros(N_VARS)
    
        resultados = {}
    
        # 7.2. RESOLUCIÓN CON MÁXIMO DESCENSO (SD)
        x_sd, f_sd, g_sd, iter_sd = Maximo_Descenso(f_wrapper, g_wrapper, x0, epsilon_g, max_iter)
        resultados['SD'] = {'x': x_sd, 'f': f_sd, 'g': g_sd, 'iter': iter_sd, 'norma_g': np.linalg.norm(g_sd)}
    
        # 7.3. RESOLUCIÓN CON NEWTON PROTEGIDO (NP)
        x_np, f_np, g_np, iter_np = Newton_Protegido(f_wrapper, g_wrapper, G_mat, x0, epsilon_g, epsilon_p, 100, 1e-4)
        resultados['NP'] = {'x': x_np, 'f': f_np, 'g': g_np, 'iter': iter_np, 'norma_g': np.linalg.norm(g_np)}
    
        # 7.4. RESOLUCIÓN CON PYOMO/IPOPT (IPOPT)
        x_ipopt, f_ipopt, g_ipopt = Resolver_Pyomo(P_COEF, Q_COEF, R_COEF, N_VARS)
        resultados['IPOPT'] = {'x': x_ipopt, 'f': f_ipopt, 'g': g_ipopt, 'norma_g': np.linalg.norm(g_ipopt)}
    
    
        # 7.5. GENERACIÓN DEL ARCHIVO DE SALIDA 
        with open(nombre_out, 'w') as f:
            
            f.write("PRACTICA  4: MINIMIZACION DE FUNCIONES DE n VARIABLES SIN RESTRICCIONES\n")
            f.write("             ==========================================================\n\n")
    
            
            # Función auxiliar para escribir una sección
            def escribir_seccion(f, titulo, datos):
                f.write(f"--- RESULTADOS: {titulo} ---\n")
                if 'iter' in datos:
                    f.write(f"Iteraciones: {datos['iter']}\n\n")
                if 'time' in datos:
                    f.write(f"Tiempo de CPU: {datos['time']:.6f}s\n\n")
    
                f.write("Minimo x y gradiente g:\n")
                f.write("=======================================\n")
                f.write(" j             x(j)                g(j)\n")
                f.write("---------------------------------------\n")
                
                for j in range(1, N_VARS + 1):
                    f.write(f" {j:2d} :: {datos['x'][j-1]:20.16f} {datos['g'][j-1]:20.16f}\n") 
    
                f.write("=======================================\n\n")
                f.write(f"fmin = {datos['f']:20.16f}\n\n")
                f.write(f"||g|| = {datos['norma_g']:20.16f}\n")
                f.write("--------------------------------------------------\n\n")
    
    
            # Escribir las 3 secciones
            escribir_seccion(f, "1. MÁXIMO DESCENSO (STEEPEST DESCENT)", resultados['SD'])
            escribir_seccion(f, "2. NEWTON PROTEGIDO", resultados['NP'])
            escribir_seccion(f, "3. PYOMO/IPOPT", resultados['IPOPT'])
    
    except Exception as e:
        print(f"\n Se produjo un error durante la ejecución. Asegúrese de tener NumPy, Pyomo y el solver IPOPT instalados.")
        print(f"Detalle: {e}")


# ===================== DATOS DE EJECUCIÓN =====================

archivo_entrada = 'Datos4.dat'
archivo_salida = 'Solucion4.sol'

epsilon_g = 1e-10
epsilon_p = 1e-12
max_iter = 5000

Ejecutar_Escribir(archivo_entrada, archivo_salida, epsilon_g, epsilon_p, max_iter)
